In [1]:
#데이터 세트와 결합: Concat과 Append
import pandas as pd
import numpy as np

In [3]:
def make_df(cols, ind):
    """빠르게 df 생성"""
    data = {c: [str(c)+str(i) for i in ind] for c in cols}
    return pd.DataFrame(data,ind)

In [7]:
df1 = make_df("AB", [1,2])
df2 = make_df("AB", [3,4])
pd.concat([df1,df2])

,A,B
1,A1,B1
2,A2,B2
3,A3,B3
4,A4,B4


In [13]:
x = make_df("AB",[0,1])
y = make_df("AB",[2,3])
y.index = x.index
print(pd.concat([x,y]))
print(pd.concat([x,y],ignore_index=True))

    A   B
0  A0  B0
1  A1  B1
0  A2  B2
1  A3  B3
    A   B
0  A0  B0
1  A1  B1
2  A2  B2
3  A3  B3


In [15]:
pd.concat([x,y], keys=["x","y"])

A   B
x 0  A0  B0
  1  A1  B1
y 0  A2  B2
  1  A3  B3

In [112]:
#미국 주 데이터 예제
pop = pd.read_csv("../data/state-population.csv")
areas = pd.read_csv("../data/state-areas.csv")
abbrevs = pd.read_csv("../data/state-abbrevs.csv")
print(pop.head(3)) ; print(areas.head(3)); print(abbrevs.head(3))

  state/region     ages  year  population
0           AL  under18  2012   1117489.0
1           AL    total  2012   4817528.0
2           AL  under18  2010   1130966.0
     state  area (sq. mi)
0  Alabama          52423
1   Alaska         656425
2  Arizona         114006
     state abbreviation
0  Alabama           AL
1   Alaska           AK
2  Arizona           AZ


In [113]:
merged = pd.merge(pop,abbrevs,how="outer",left_on = "state/region", right_on = "abbreviation")
merged = merged.drop("abbreviation",1)
merged.isnull().any() #열단위로 작동

state/region    False
ages            False
year            False
population       True
state            True
dtype: bool

In [114]:
print(merged[merged["population"].isnull()].head()) # 2000sus dlwjs푸에르토리코 인구정보 없어서 그럼
print(merged[merged["population"].isnull()].tail())

     state/region     ages  year  population state
2448           PR  under18  1990         NaN   NaN
2449           PR    total  1990         NaN   NaN
2450           PR    total  1991         NaN   NaN
2451           PR  under18  1991         NaN   NaN
2452           PR    total  1993         NaN   NaN
     state/region     ages  year  population state
2463           PR    total  1998         NaN   NaN
2464           PR    total  1997         NaN   NaN
2465           PR  under18  1997         NaN   NaN
2466           PR    total  1999         NaN   NaN
2467           PR  under18  1999         NaN   NaN


In [115]:
merged.loc[merged["state"].isnull(),"state/region"].unique() #약어키 등장 X

array(['PR', 'USA'], dtype=object)

In [116]:
merged.loc[merged["state/region"] == "PR", "state"] = "Puerto Rico"
merged.loc[merged["state/region"] == "USA", "state"] = "United States"
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state           False
dtype: bool

In [117]:
final = pd.merge(merged, areas, on="state", how="left")
final.isnull().any()

state/region     False
ages             False
year             False
population        True
state            False
area (sq. mi)     True
dtype: bool

In [118]:
final["state"][final["area (sq. mi)"].isnull()].unique()

array(['United States'], dtype=object)

In [119]:
final.dropna(inplace=True) #어차피 미국 전체값 의논 필요 크게 X

In [120]:
import numexpr
data2010 = final.query("year == 2010 & ages == 'total'")
data2010.head()

,state/region,ages,year,population,state,area (sq. mi)
3,AL,total,2010,4785570.0,Alabama,52423.0
91,AK,total,2010,713868.0,Alaska,656425.0
101,AZ,total,2010,6408790.0,Arizona,114006.0
189,AR,total,2010,2922280.0,Arkansas,53182.0
197,CA,total,2010,37333601.0,California,163707.0


In [121]:
data2010.set_index('state',inplace=True)
density = data2010["population"] / data2010["area (sq. mi)"]
density.sort_values(ascending=False, inplace=True)
density.head()

state
District of Columbia    8898.897059
Puerto Rico             1058.665149
New Jersey              1009.253268
Rhode Island             681.339159
Connecticut              645.600649
dtype: float64

In [122]:
density.tail()

state
South Dakota    10.583512
North Dakota     9.537565
Montana          6.736171
Wyoming          5.768079
Alaska           1.087509
dtype: float64